In [1]:
import glob
import time
from bs4 import BeautifulSoup
import pandas
print("pandas",pandas.__version__)

pandas 1.3.3


In [2]:
html_files = glob.glob("sample_html_files/*.html")

In [3]:
html_files

['sample_html_files/file1.html',
 'sample_html_files/file2.html',
 'sample_html_files/file3.html',
 'sample_html_files/file4.html',
 'sample_html_files/file5.html']

In [4]:
with open(html_files[0], 'r') as file_handle:
    file_content = file_handle.read()

In [14]:
soup = BeautifulSoup(file_content, 'html.parser')
table = soup.table

In [15]:
table

<table border="1">
<tr>
<th>key</th>
<th>val</th>
</tr><tr>
<td>parent</td>
<td><a href="file3.html">file3</a></td>
</tr><tr>
<td>child(ren)</td>
<td>None</td>
</tr><tr>
<td>links</td>
<td>
<ul>
<li>read from/write to <a href="file2.html">file2</a></li>
<li>write to <a href="file4.html">file4</a></li>
</ul>
</td>
</tr><tr>
</tr></table>

In [16]:
# the pandas read_html saves only plain text (no hyperlinks, no lists)
# https://stackoverflow.com/a/56510975/1164295
# https://pandas.pydata.org/docs/user_guide/io.html#io-read-html
df = pandas.read_html(str(table))
df

[          key                                      val
 0      parent                                    file3
 1  child(ren)                                     None
 2       links  read from/write to file2 write to file4
 3         NaN                                      NaN]

In [20]:
# modified version of https://stackoverflow.com/a/53775331/1164295

column_names = ['key','val']

data = []
for row in table.find_all('tr'):
    row_data = []
    for td in row.find_all('td'):
        list_check = td.find("ul")
        if list_check is not None: # a list is present
            for li in td.find_all('li'):
                print('li=',li)
                link_check = li.find('a')
                if link_check is not None:  # there is a hyperlink
                    link = li.a['href']
        else: # no list
            td_check = td.find('a')
            if td_check is not None: # there is a hyperlink
                link = td.a['href']
                row_data.append(link)
            else: # no link present
                not_link = ''.join(td.stripped_strings)
                if not_link == '':
                     not_link = None
                row_data.append(not_link)

    data.append(row_data)
df = pandas.DataFrame(data[1:], columns=column_names)
df_dict = df.to_dict('records')

li= <li>read from/write to <a href="file2.html">file2</a></li>
li= <li>write to <a href="file4.html">file4</a></li>


In [19]:
df

,key,val
0,parent,file3.html
1,child(ren),None
2,links,file2.html
3,None,None
